In [1]:
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium import plugins
from folium.plugins import HeatMap, MarkerCluster
from folium import IFrame
import ipywidgets
#import geopy
#import geocoder

import numpy as np
import pandas as pd

# Mapping and Visualization

## CENTROS MEDICOS EN MADRID

1. COORDENADA-X: coordenada de 6 caracteres sobre plano de la ciudad.

    
2. COORDENADA-Y: coordenada de 7 caracteres sobre plano de la ciudad.

    * Ambas coordenadas, UTM - 30, están proyectadas en el sistema de referencia ETR89. 
    * Actualizado a mayo 2019


3. LATITUD: coordenada latitud en el sistema de referencia WGS84

    
4. LONGITUD: coordenada longitud en el sistema de referencia WGS84

    * La coordenadas longitud y latitud, si se abre el fichero CSV con Excel, las muestra mal por un tema de conversión de EXCEL y manejo de puntos y comas, como separadores. Estos campos, si se abre con otro software, las muestra sin problemas. También está disponible la versión en XML, que los muestra sin problemas ni conversiones. 

In [2]:
centros_medicos_clean = pd.read_csv("../data/centros_medicos_clean.csv")
centros_medicos_clean.head()

,Unnamed: 0,NOMBRE,DIRECCION,DISTRITO,COD_POSTAL,COORDENADA-X,COORDENADA-Y,LATITUD,LONGITUD,TIPO
0,0,Centro Concertado de Atención a las Adicciones...,CALLE SANTA HORTENSIA 1,CHAMARTIN,28002,443159,4477482,40.446053,-3.670277,CentrosAsistenciaDrogodependientes
1,1,Centro Concertado de Atención a las Adicciones...,AVENIDA PORTUGAL 0,LATINA,28011,438577,4474219,40.416331,-3.723991,CentrosAsistenciaDrogodependientes
2,2,Centro de Atención a las Adicciones (CAD) de A...,PASEO CHOPERA 12,ARGANZUELA,28045,440757,4471619,40.393057,-3.698067,CentrosAsistenciaDrogodependientes
3,3,Centro de Atención a las Adicciones (CAD) de H...,CALLE MINAYA 7,HORTALEZA,28043,446072,4479929,40.468289,-3.636150,CentrosAsistenciaDrogodependientes
4,4,Centro de Atención a las Adicciones (CAD) de L...,CALLE CAMARENA 10,LATINA,28047,435820,4471982,40.395971,-3.756272,CentrosAsistenciaDrogodependientes


In [3]:
centros_medicos_clean['TIPO'].unique()

array(['CentrosAsistenciaDrogodependientes', 'OtrosCentrosMedicos',
       'CentrosEspecialidadesMedicas', 'CentrosSalud',
       'CentrosRehabilitacionPsicosocial',
       'CentrosPrevencionEnfermedades', 'CentrosSaludMental',
       'HospitalesClinicasSanatorios'], dtype=object)

In [4]:
max_lat = centros_medicos_clean['LATITUD'].max()
max_lat

40.540270273154896

In [5]:
max_lon = centros_medicos_clean['LONGITUD'].max()
max_lon

-3.57810735476017

### Mapa con markers

In [6]:
map_centros = Map(location=[max_lat, max_lon],zoom_start=10)
map_centros

In [7]:
for i,row in centros_medicos_clean.iterrows():
    
    tipos = {"location" : [row["LATITUD"], row["LONGITUD"]],
                "tooltip" : row["TIPO"]}
    
    if row["TIPO"] == "CentrosAsistenciaDrogodependientes":
        icon = Icon( color = "white",
                    prefix = "fa",
                    icon = "heartbeat",
                    icon_color = "red"
        )

    elif row["TIPO"] == "OtrosCentrosMedicos":
        icon = Icon( color = "blue",
                    prefix = "fa",
                    icon = "plus-square",
                    icon_color = "white"
        )
     
    elif row["TIPO"] == "CentrosEspecialidadesMedicas":
        icon = Icon( color = "white",
                    prefix = "fa",
                    icon = "medkit",
                    icon_color = "black"
        )
    
    elif row["TIPO"] == "CentrosSalud":
        icon = Icon( color = "white",
                    prefix = "fa",
                    icon = "h-square",
                    icon_color = "blue"
        )
    
    elif row["TIPO"] == "CentrosRehabilitacionPsicosocial":
        icon = Icon( color = "red",
                    prefix = "fa",
                    icon = "heart-o",
                    icon_color = "white"
        )
    
    elif row["TIPO"] == "CentrosPrevencionEnfermedades":
        icon = Icon( color = "white",
                    prefix = "fa",
                    icon = "stethoscope",
                    icon_color = "green"
        )
        
    elif row["TIPO"] == "CentrosSaludMental":
        icon = Icon( color = "white",
                    prefix = "fa",
                    icon = "user-md",
                    icon_color = "black"
        )
    
    # HospitalesClinicasSanatorios
    else:
        icon = Icon(color = "white",
                    prefix = "fa",
                    icon = "hospital-o",
                    icon_color = "gray"
        )
    
    Marker (**tipos,icon = icon).add_to(map_centros)


map_centros

* CAMBIAR ICONOS POR LOS QUE TIENES GUARDADOS!!

### Mapa de calor

In [8]:
# Variables 

Drogodependientes = centros_medicos_clean[centros_medicos_clean.TIPO == "CentrosAsistenciaDrogodependientes"]

Otros = centros_medicos_clean[centros_medicos_clean.TIPO == "OtrosCentrosMedicos"]

Centros_especialidades = centros_medicos_clean[centros_medicos_clean.TIPO == "CentrosEspecialidadesMedicas"]

Centros_salud = centros_medicos_clean[centros_medicos_clean.TIPO == "CentrosSalud"]

Rehabilitacion_Psicosocial = centros_medicos_clean[centros_medicos_clean.TIPO == "CentrosRehabilitacionPsicosocial"]

Prevencion_Enfermedades = centros_medicos_clean[centros_medicos_clean.TIPO == "CentrosPrevencionEnfermedades"]

Salud_mental = centros_medicos_clean[centros_medicos_clean.TIPO == "CentrosSaludMental"]

Hospitales = centros_medicos_clean[centros_medicos_clean.TIPO == "HospitalesClinicasSanatorios"]

In [9]:
map_calor = Map(location=[max_lat, max_lon],zoom_start=10)

In [10]:
centros_medicos_clean['TIPO'].unique()

array(['CentrosAsistenciaDrogodependientes', 'OtrosCentrosMedicos',
       'CentrosEspecialidadesMedicas', 'CentrosSalud',
       'CentrosRehabilitacionPsicosocial',
       'CentrosPrevencionEnfermedades', 'CentrosSaludMental',
       'HospitalesClinicasSanatorios'], dtype=object)

In [11]:
Drogodependientes = centros_medicos_clean[centros_medicos_clean.TIPO == "CentrosAsistenciaDrogodependientes"]
Drogodependientes_group = folium.FeatureGroup(name = "Centros_Asistencia_Drogodependientes")
HeatMap(data = Drogodependientes[["LATITUD","LONGITUD"]], radius = 15).add_to(Drogodependientes_group)
Drogodependientes_group.add_to(map_calor)
map_calor

In [12]:
OtrosCentrosMedicos = centros_medicos_clean[centros_medicos_clean.TIPO == "OtrosCentrosMedicos"]
OtrosCentrosMedicos_group = folium.FeatureGroup (name = "OtrosCentrosMedicos")
HeatMap(data = OtrosCentrosMedicos[["LATITUD","LONGITUD"]], radius = 15).add_to(OtrosCentrosMedicos_group)
OtrosCentrosMedicos_group.add_to(map_calor)
folium.LayerControl(collapsed = False).add_to(map_calor)


map_calor

* COMO METOS MAS LAYEERSSSSS!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [14]:
CentrosEspecialidadesMedicas = centros_medicos_clean[centros_medicos_clean.TIPO == "CentrosEspecialidadesMedicas"]
CentrosEspecialidadesMedicas_group = folium.FeatureGroup(name = "CentrosEspecialidadesMedicas")
HeatMap(data = CentrosEspecialidadesMedicas[["LATITUD","LONGITUD"]], radius = 15).add_to(CentrosEspecialidadesMedicas_group)
CentrosEspecialidadesMedicas_group.add_to(map_calor)
folium.LayerControl(collapsed = False).add_to(map_calor)

map_calor

In [ ]:
'''
SALE LA IMAGEN PERO NO EN DONDE TIENE QUE SALIR UBICADA!!

map_icons = Map(location=[max_lat, max_lon],zoom_start=10)

img_incon = folium.raster_layers.ImageOverlay(name = 'icons',
                                            image = '../icons/Hospital-clinica-sanatorio-icon.png',
                                            bounds = [max_lat, max_lon],
                                            opacity = 0.8)

img_incon.add_to(map_icons)

map_icons

'''

## Farmacias en Madrid

In [94]:
farmacias_clean = pd.read_csv("../data/farmacias_clean.csv")
farmacias_clean.head()

,Unnamed: 0,titular_nombre,direccion,municipio_nombre,localizacion_coordenada_x,localizacion_coordenada_y,latitud,longitud
0,0,Amaya Corral Palomino,PLAZA del Dos de Mayo 6,Madrid,440267.0,4475426.0,40.427326,-3.704186
1,1,Montserrat Gimenez Mestre,CALLE de Ríos Rosas 50,Madrid,440954.0,4477004.0,40.441590,-3.696234
2,2,Justo Aquilino Corral Aragon,CALLE de Argensola 12,Madrid,441063.0,4475193.0,40.425284,-3.694781
3,3,Irene Arroita Del Campo,CALLE de Alcalá 98,Madrid,442754.0,4475148.0,40.424996,-3.674844
4,4,Esther Sansi Ponsico,PLAZA de Salesas 7,Madrid,441090.0,4475021.0,40.423736,-3.694447


In [95]:
max_latitud = farmacias_clean.latitud.max()
max_latitud

41.059912104960105

In [96]:
max_longitud = farmacias_clean.longitud.max()
max_longitud

-3.1054510767518786

In [97]:
map_farmacias = Map(location=[max_latitud, max_longitud],zoom_start=10)
map_farmacias